In [1]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine, text

In [3]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [5]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.30
Running PyMongo Version: 4.10.1


In [7]:
mysql_args = {
    "uid" : "root",
    "pwd" : "monty2017!",
    "hostname" : "localhost",
    "dbname" : "sakila_db"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "fwx5ax",
    "password" : "lH1tx43vqHYeRMbW",
    "cluster_name" : "Cluster0",
    "cluster_subnet" : "tjhne",
    "cluster_location" : "atlas", # "local"
    "db_name" : "sakila_db"
}
uid = "root"
pwd = "monty2017!"
hostname = "localhost"
dbname = "sakila_db"

In [9]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [11]:
conn_str = f"mysql+pymysql://{uid}:{pwd}@{hostname}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dbname}`;"))
connection.execute(text(f"USE {dbname};"))

connection.close()

In [13]:
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"inventory" : 'Inventory_Table.json'
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)   

Create and Populate New Inventory Dimension Table

In [15]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "inventory"

df_inventory = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_inventory.head(2)

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17


In [16]:
df_inventory.drop(['last_update'], axis=1, inplace=True)

In [17]:
dataframe = df_inventory
table_name = 'dim_inventory'
primary_key = 'inventory_id'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key,db_operation, **mysql_args)
df_inventory.head(5)

,inventory_id,film_id,store_id
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,2


Upload CSV for Payment Table

In [20]:
data_dir = os.path.join(os.getcwd(), 'data')
data_file = os.path.join(data_dir, 'payment_table.csv')

df_payment = pd.read_csv(data_file, header=0, index_col=0)
df_payment.head()

,customer_id,staff_id,rental_id,amount,payment_date,last_update
payment_id,,,,,,
1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30
3,1,1,1185,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30
4,1,2,1422,0.99,2005-06-15 18:02:53,2006-02-15 22:12:30
5,1,2,1476,9.99,2005-06-15 21:08:46,2006-02-15 22:12:30


In [21]:
df_payment.drop(['last_update'], axis=1, inplace=True)
df_payment.columns
df_payment.head()

,customer_id,staff_id,rental_id,amount,payment_date
payment_id,,,,,
1,1,1,76,2.99,2005-05-25 11:30:37
2,1,1,573,0.99,2005-05-28 10:35:23
3,1,1,1185,5.99,2005-06-15 00:54:12
4,1,2,1422,0.99,2005-06-15 18:02:53
5,1,2,1476,9.99,2005-06-15 21:08:46


Create Primary Key

In [28]:
df_payment.insert(0, "payment_key", range(1,  df_payment.shape[0]+1))

Load Payment Table into Database

In [31]:
dataframe = df_payment
table_name = 'dim_payment'
primary_key = 'payment_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key,db_operation, **mysql_args)

In [ ]:
Transform Date Dimension - DO NOT RUN UNTIL FIRST SECTION OF SQL FILE HAS BEEN RUN

In [33]:
sql_dim_date = "SELECT date_key, full_date FROM sakila_db.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02
